In [4]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
%matplotlib inline

In [4]:
"""url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')"""

'url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"\n\ndataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,\n                                    untar=True, cache_dir=\'.\',\n                                    cache_subdir=\'\')\n\ndataset_dir = os.path.join(os.path.dirname(dataset), \'aclImdb\')'

In [5]:
data = pd.read_csv('../data/factnameh_2010_10_01.csv')
data.head()

,title,claim_summary,tags,org,entity,entity_role,date,body_text,conclusion_sum,conclusion,links,url
0,واکسیناسیون و پایان کرونا در چین؟,کرونا به دلیل واکسیناسیون در چین پایان یافته است.,NaN,social-media,شبکه‌های اجتماعی,\n تلگرام، توییتر، ...,۱۳۹۹/۷/۷,سوالات درباره وضعیت کرونا در چین، یکی از سوژه‌...,false,\n گفته یا آمار، نادرست است یا دست‌کم سندی...,https://www.instagram.com/p/CE_QEq-H6y3/;;http...,https://factnameh.com/fact-checks/2020-09-28-c...
1,آیا ۷۰ درصد تورم به دلیل ناتوانی دولت است و رب...,۷۰درصد تورم و گرانی ریشه در ناتوانی دولت دارد،...,NaN,ali-nazari,علی نظری,\n روزنامه‌نگار و ن...,۱۳۹۹/۷/۵,یکی از مخاطبان فکت‌نامه در توییتر از ما خواسته...,false,\n گفته یا آمار، نادرست است یا دست‌کم سندی...,https://twitter.com/nazary43/status/1308482215...,https://factnameh.com/fact-checks/2020-09-26-i...
2,بیل گیتس و کاهش جمعیت جهان با واکسن؟,بیل گیتس می‌خواهد از واکسن‌ها برای کاهش جمعیت ...,NaN,press,مطبوعات و رسانه‌ها,\n نشریات چاپی و آن...,۱۳۹۹/۷/۳,ادعاهای مربوط به خطرناک بودن واکسیناسیون، نه ت...,outrageous,\n متاسفانه مرورگر شما قابلیت پخش تصویر ندا...,https://www.tasnimnews.com/fa/news/1399/06/31/...,https://factnameh.com/fact-checks/2020-09-24-b...
3,جوشانده سعال و درمان کرونا,جوشانده سعال داروی درمان کرونا است.,NaN,press,مطبوعات و رسانه‌ها,\n نشریات چاپی و آن...,۱۳۹۹/۷/۱,تلاش‌های متخصصان از چین و روسیه تا بریتانیا و ...,false,\n گفته یا آمار، نادرست است یا دست‌کم سندی...,https://factnameh.com/fact-checks/2020-08-05-b...,https://factnameh.com/fact-checks/2020-09-22-c...
4,آیا خسارت لغو سهمیه‌بندی بنزین در سال ۹۴، بیش ...,خسارت لغو سهمیه‌بندی بنزین در سال ۹۴ بیش از ۱۵...,NaN,alireza-zakani,علیرضا زاکانی,\n رئیس مرکز پژوهش‌...,۱۳۹۹/۶/۲۹,علیرضا زاکانی، نماینده قم و رئیس مرکز پژوهش ها...,no_data,\n گفته یا آمار را نمی‌توان با فکت‌های قابل...,https://rc.majlis.ir/fa/news/show/1607846;;htt...,https://factnameh.com/fact-checks/2020-09-19-i...


In [8]:
poems = pd.read_csv('../data/poems/poems.tsv',sep = '\t')
poems

,poet,century,poem
0,رودکی,3,گر من این دوستی تو ببرم تا لب گور بزنم نعره...
1,رودکی,3,به حق نالم ز هجر دوست زارا سحرگاهان چو بر گ...
2,رودکی,3,به نام نیک تو، خواجه، فریفته نشوم که نام نی...
3,رودکی,3,دلا، تا کی همی جویی منی را؟ چه داری دوست هر...
4,رودکی,3,گرفت خواهم زلفین عنبرین ترا به بوسه نقش‌کنم...
...,...,...,...
54519,اِ لیـــار (جبار محمدی ),15,وطن خورشید آمال است. مرا مامی کهن سال است. ...
54520,اِ لیـــار (جبار محمدی ),15,در آسمان این خطّه همیشه خورشید است فرازِ...
54521,اِ لیـــار (جبار محمدی ),15,عمر است چون خزانه هر لحظه اش چو دُرّی است ...
54522,اِ لیـــار (جبار محمدی ),15,بهار عمر ما دور جوانی است پر از باران عشق و...


In [50]:
vocab_size = 1000
embedding_dim = 32
max_length = 100
trunc_type = 'post'
padding_type = 'post'
oov_tok  = "<OOV>"
training_size = 40000
num_epochs = 5

سطرهای دیتاست را به صورت تصادفی بر می زنیم.

In [19]:
poems = poems.sample(frac = 1) 

In [20]:
poems.head()

,poet,century,poem
27828,سلمان ساوجی,8,شنودم که می‌گفت بشوده به شیخ که احوال حاجی...
44146,صائب تبریزی,11,هر کس که در نماز به روی و ریا رود بر پشت با...
10639,عطار,6,ای خلق چرا در تب و تفتید آخر نابوده و ناآمد...
29281,شاه نعمت‌الله ولی,8,رنج غربت تو از غریبان پرس دردمندی ز دردمندا...
14235,عطار,6,آن مجرد باطن و ظاهر، آن مسافر غایب و حاضر، آن ...


In [95]:
poems['poet_cat'] = poems['poet'].astype('category').cat.codes

In [96]:
poems_text = poems.poem.values.tolist()
poems_text[1]

'هر کس که در نماز به روی و ریا رود    بر پشت بام کعبه به کسب هوا رود    بر عشق رهروی که کند عقل اختیار    از خضر بگسلد ز پی نقش پا رود    تا باز می کنند نظر بسته می شود    از هر دری که اهل طلب بینوا رود    این قفل وا شود به کلید شکستگی    هردانه ای که نرم شد از آسیا رود    بینا کسی بود که نهد پا به احتیاط    در وادیی که کوردر او بی عصا رود    خواب غرور لازم ارباب دولت است    کی تیرگی ز سایه بال هما رود    بیرون نرفت سرمه به شستن ز چشم یار    دود از سیاه خانه لیلی کجا رود    نادان شود ز اهل بصیرت به خاکمال    کوتاه دیدگی اگر از توتیا رود    بی مغز را ز جای برد گفتگوی پوچ    کاه سبک عنان ز پی کهربا رود    هر کس هرآنچه یافته زین خاک یافته است    از آستان میکده صائب کجارود'

In [97]:
labels = poems.poet_cat.values.tolist()
labels[1]

33

حالا بیایید ببینم که چه تعداد شاعر در دیتاست ما حضور دارند یا به عبارتی دیگر مساله دستی بندی ما چند کلاس دارد.

In [111]:
len(set(labels))

67

In [98]:
training_poems = poems_text[:training_size]
training_labels = labels[:training_size]
test_poems = poems_text[training_size:]
test_labels = labels[training_size:]


In [99]:
tokenizer = Tokenizer(num_words=vocab_size,
                     oov_token = oov_tok)

In [100]:
tokenizer.fit_on_texts(training_poems)

In [101]:
word_index = tokenizer.word_index
#word_index

In [102]:
training_sequences = tokenizer.texts_to_sequences(training_poems)

In [103]:
training_sequences[1][:20]

[20,
 70,
 4,
 6,
 822,
 5,
 49,
 2,
 1,
 299,
 10,
 311,
 702,
 630,
 5,
 1,
 362,
 299,
 10,
 42]

In [104]:
training_poems[1][:20]

'هر کس که در نماز به '

In [106]:
training_padded = pad_sequences(training_sequences,
                                maxlen=max_length,
                                padding=padding_type,
                               truncating=trunc_type)

In [107]:
type(training_padded)

numpy.ndarray

In [108]:
training_poems[2]

'ای خلق چرا در تب و تفتید آخر    نابوده و ناآمده رفتید آخر    ای بیخبران این در و درگاه عظیم    خالی مگذارید و مخفتید آخر'

In [109]:
training_padded[2]

array([ 26, 147, 167,   6,   1,   2,   1, 137,   1,   2,   1,   1, 137,
        26,   1,  14,   6,   2,   1,   1, 688,   1,   2,   1, 137,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0])

In [110]:
test_sequences = tokenizer.texts_to_sequences(test_poems)
test_padded = pad_sequences(test_sequences,
                             maxlen=max_length,
                                padding=padding_type,
                               truncating=trunc_type)

In [116]:
training_labels = np.array(training_labels).reshape(-1,1) 
test_labels = np.array(test_labels) 

In [117]:
training_labels[1:10]

array([[33],
       [39],
       [28],
       [39],
       [33],
       [ 6],
       [ 9],
       [58],
       [39]])

In [88]:
training_padded.shape

(40000, 100)

In [112]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,
                                  embedding_dim,
                                 input_length = max_length),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                             tf.keras.layers.Dense(32,activation = 'relu'),
                             tf.keras.layers.Dense(67,activation = 'softmax')])

In [113]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 32)           32000     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64)                16640     
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 67)                2211      
Total params: 52,931
Trainable params: 52,931
Non-trainable params: 0
_________________________________________________________________


In [114]:
model.compile(loss= 'sparse_categorical_crossentropy',
             optimizer='adam',
             metrics = ['accuracy'])

In [118]:
history = model.fit(training_padded,training_labels,
                    validation_split=0.1,
                    epochs=num_epochs,
                    batch_size=32,
                   verbose=1)

Epoch 1/5
1125/1125 [==============================] - 33s 29ms/step - loss: 2.9622 - accuracy: 0.2678 - val_loss: 2.5308 - val_accuracy: 0.3417
Epoch 2/5
1125/1125 [==============================] - 33s 30ms/step - loss: 2.4358 - accuracy: 0.3626 - val_loss: 2.3059 - val_accuracy: 0.3860
Epoch 3/5
1125/1125 [==============================] - 38s 34ms/step - loss: 2.1839 - accuracy: 0.4133 - val_loss: 2.1525 - val_accuracy: 0.4200
Epoch 4/5
1125/1125 [==============================] - 37s 33ms/step - loss: 2.0407 - accuracy: 0.4498 - val_loss: 2.0638 - val_accuracy: 0.4532
Epoch 5/5
1125/1125 [==============================] - 33s 30ms/step - loss: 1.9186 - accuracy: 0.4841 - val_loss: 1.9704 - val_accuracy: 0.4765


### 
حالا بیاید معماری مدل شبکه عصبی را تغییر دهیم.


### پیش پردازش بیشتر داده ها
یک کار دیگری که می توانیم انجام بدهیم این است که کلمات توقف را حذف کنیم.


In [2]:
with open('../data/poems/stop_words.txt','r',encoding='utf8') as f:
    stop_words = f.read().split('\n')

In [3]:
stop_words[1]

'ولیکن'

In [ ]:
### تمرین
